In [1]:
from langchain import document_loaders as dl
from langchain import embeddings
from langchain import text_splitter as ts
from langchain import vectorstores as vs
import torch
from transformers import pipeline
from langchain.prompts import PromptTemplate
from langchain.schema import StrOutputParser
from langchain.schema.runnable import RunnablePassthrough
from langchain.schema.runnable import RunnableParallel
from langchain.prompts import PromptTemplate
from operator import itemgetter
import time
import re
from langchain_huggingface import HuggingFacePipeline

c:\Users\Carlini\miniconda3\envs\llama\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def split_doc(path, chunk_size=500, chunk_overlap=20):
    loader = dl.PyPDFLoader(path)
    doc = loader.load()
    text_splitter = ts.RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)
    doc_splitted = text_splitter.split_documents(documents=doc)
    return doc_splitted

In [3]:
doc_path = 'C:/Users/Carlini/Desktop/Gianluca/quantum-computing-an-emerging-ecosystem.pdf'

In [4]:
document_split = split_doc(doc_path)

In [5]:
for doc in document_split:
    print(doc)

page_content='Quantum computing:  
An emerging ecosystem and 
industry use cases
December 2021' metadata={'source': 'C:/Users/Carlini/Desktop/Gianluca/quantum-computing-an-emerging-ecosystem.pdf', 'page': 0}
page_content='McKinsey & Company
McKinsey & Company is a global management consulting firm, deeply committed to helping institutions in the 
private, public, and social sectors achieve lasting success. For more than 90 years, our primary objective has been 
to serve as our clients’ most trusted external adviser. With consultants in more than 100 cities in over 60 markets, 
across industries and functions, we bring unparalleled expertise to clients all over the world. We work closely with' metadata={'source': 'C:/Users/Carlini/Desktop/Gianluca/quantum-computing-an-emerging-ecosystem.pdf', 'page': 1}
page_content='teams at all levels of an organization to shape winning strategies, mobilize for change, build capabilities, and drive 
successful execution.
Cover image: © Paolo Carnassal

In [6]:
from sentence_transformers import SentenceTransformer

model_kwargs = {'device': 'cuda:0'}
encode_kwargs = {'normalize_embeddings': False}

model = "sentence-transformers/all-mpnet-base-v2"

embedding_model = embeddings.HuggingFaceBgeEmbeddings(model_name=model, model_kwargs=model_kwargs, encode_kwargs=encode_kwargs)

In [7]:
def create_db(docs, model):
    model_vectorstore = vs.FAISS
    db = None

    try:
        content = []
        metadata = []

        for doc in docs:
            content.append(doc.page_content)
            metadata.append({"source": doc.metadata})

        db = model_vectorstore.from_texts(content, embedding_model, metadata)
    
    except Exception as e:
        print(e)

    return db

In [8]:
db = create_db(document_split, model)

In [9]:
db

In [10]:
query = "What is quantum computing?"
retriever = db.as_retriever(search_type="similarity_score_threshold", search_kwargs={"k": 6, 'score_threshold': 0.01})
retrieved_docs = retriever.get_relevant_documents(query)

C:\Users\Carlini\AppData\Local\Temp\ipykernel_34000\2892541505.py:3: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  retrieved_docs = retriever.get_relevant_documents(query)


In [18]:
retrieved_docs

[Document(metadata={'source': {'source': 'C:/Users/Carlini/Desktop/Gianluca/quantum-computing-an-emerging-ecosystem.pdf', 'page': 35}}, page_content='quantum-computing degree programs—and update their STEM programs—to meet the need for quantum-\ncomputing talent.\n34 For more details, see “The Quantum Technology Monitor,” September 2021, McKinsey.com.\n35 “Information technology – Quantum computing – Terminology and vocabulary,” ISO, November 23, 2021, iso.org; “P7130 – Standard for \nquantum technologies definitions,” The Institute of Electrical and Electronics Engineers (IEEE),  November 23, 2021, standards.ieee.org;'),
 Document(metadata={'source': {'source': 'C:/Users/Carlini/Desktop/Gianluca/quantum-computing-an-emerging-ecosystem.pdf', 'page': 11}}, page_content='Open questions in quantum computing\nExperts continue to explore many open \nquestions and debates on topics such as \nthe state of development of the technology, \nstandards and metrics for performance, and \nthe value 

In [12]:
llama = pipeline(
    "text-generation",
    model="meta-llama/Llama-3.2-1B-Instruct",
    torch_dtype=torch.float16,
    device='cuda:0',
    max_new_tokens=10000,
)

llm = HuggingFacePipeline(pipeline=llama, model_kwargs={'temperature': 0.})

In [13]:
template = """Use the following pieces of context to answer the question at the end.
If you don't know the answer, just say that you don't know, don't try to make up an answer.
{context}
Question: {question}
Helpful Answer:"""

rag_prompt_custom = PromptTemplate.from_template(template)

In [14]:
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

In [15]:
rag_chain_from_docs = (
    {
        "context": lambda input: format_docs(input["documents"]),
        "question": itemgetter("question"),
    }
    | rag_prompt_custom
    | llm
    | StrOutputParser()
)

#Second chain to postprocess the answer
rag_chain_with_source = RunnableParallel(
    {"documents": retriever, "question": RunnablePassthrough()}
) | {
    "documents": lambda input: [doc.metadata for doc in input["documents"]],
    "answer": rag_chain_from_docs,
}

In [16]:
t0=time.time()
resp = rag_chain_with_source.invoke(query)
if len(resp['documents'])==0:
  print('No documents found')
else:
  stripped_resp = re.sub(r"\n+$", " ", resp['answer'])
  print(stripped_resp)
  print('Sources',resp['documents'])
  print('Response time:', time.time()-t0)

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Use the following pieces of context to answer the question at the end.
If you don't know the answer, just say that you don't know, don't try to make up an answer.
quantum-computing degree programs—and update their STEM programs—to meet the need for quantum-
computing talent.
34 For more details, see “The Quantum Technology Monitor,” September 2021, McKinsey.com.
35 “Information technology – Quantum computing – Terminology and vocabulary,” ISO, November 23, 2021, iso.org; “P7130 – Standard for 
quantum technologies definitions,” The Institute of Electrical and Electronics Engineers (IEEE),  November 23, 2021, standards.ieee.org;

Open questions in quantum computing
Experts continue to explore many open 
questions and debates on topics such as 
the state of development of the technology, 
standards and metrics for performance, and 
the value of different use cases. 
Some debates in these areas relate to 
semantics—for example, using the term 
“circuit model for quantum computing” or 
“ga

In [17]:
resp['answer']

"Use the following pieces of context to answer the question at the end.\nIf you don't know the answer, just say that you don't know, don't try to make up an answer.\nquantum-computing degree programs—and update their STEM programs—to meet the need for quantum-\ncomputing talent.\n34 For more details, see “The Quantum Technology Monitor,” September 2021, McKinsey.com.\n35 “Information technology – Quantum computing – Terminology and vocabulary,” ISO, November 23, 2021, iso.org; “P7130 – Standard for \nquantum technologies definitions,” The Institute of Electrical and Electronics Engineers (IEEE),  November 23, 2021, standards.ieee.org;\n\nOpen questions in quantum computing\nExperts continue to explore many open \nquestions and debates on topics such as \nthe state of development of the technology, \nstandards and metrics for performance, and \nthe value of different use cases. \nSome debates in these areas relate to \nsemantics—for example, using the term \n“circuit model for quantum c